In [1]:
%load_ext autoreload
%autoreload 2

import torch
torch.__version__

'1.6.0'

In [2]:
import numpy as np
import torch
import random
import pickle
import sys
import os
import glob
import pdb
sys.path.insert(0, '/home/jupyter/VLP/pythia')
sys.path.insert(0, '/home/jupyter/VLP/')

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTrainingLossMask
from pytorch_pretrained_bert.optimization import BertAdam
from pathlib import Path
import pandas as pd
from vlp.loader_utils import batch_list_to_batch_tensors
import vlp.seq2seq_loader as seq2seq_loader
from vlp.seq2seq_loader import truncate_tokens_pair
import PIL
from vlp.lang_utils import language_eval

from fastai.vision.all import *


import pythia.tasks.processors as pythia_proc
import util
from util import *
from vlp_processor import PreprocessVLP


In [3]:
model_recover_path = '/home/jupyter/VLP/checkpoints/vqa2_g2_lr2e-5_batch512_ft_from_s0.75_b0.25/model.19.bin'
model_recover = torch.load(model_recover_path)

In [4]:
def new_model():
    cls_num_labels = 2
    type_vocab_size = 6
    relax_projection = 4
    task_idx_proj = 3
    drop_prob = .25

    model = BertForPreTrainingLossMask.from_pretrained(
                    'bert-base-cased', state_dict=model_recover, num_labels=cls_num_labels,
                    type_vocab_size=type_vocab_size, relax_projection=relax_projection,
                    config_path=None, task_idx=task_idx_proj,
                    max_position_embeddings=None, 
                    label_smoothing=0,
                    fp32_embedding=False,
                    cache_dir='./.pretrained_model',
                    drop_prob=drop_prob, 
                    enable_butd=True,
                    len_vis_input=max_alen,
                    tasks='img2txt')
    return model.cuda()

In [5]:
tokenizer = get_tokenizer()
max_alen = 100
max_blen = 100
max_seq_len = max_alen + max_blen + 3
tokenizer.max_len = max_seq_len


In [6]:
max_pred = 3
mask_prob = .15
vis_mask_prob=0
mask_image_regions = vis_mask_prob > 0
truncate_config={
    'max_len_b': max_blen, 'trunc_seg': 'b', 'always_truncate_tail': True}
region_pref = HATE_FEAT_PATH / 'feat_cls_1000/hateful_vlp_checkpoint_trainval'
bbox_pref = HATE_FEAT_PATH / 'raw_bbox/hateful_vlp_checkpoint_trainval'
id_digits=2
proc=PreprocessVLP(max_pred, mask_prob,
            list(tokenizer.vocab.keys()), tokenizer.convert_tokens_to_ids, max_seq_len,
            truncate_config=truncate_config, mask_image_regions=mask_image_regions,
            mode="bi", len_vis_input=max_alen,
            vis_mask_prob=vis_mask_prob, 
            region_bbox_prefix=str(bbox_pref), region_det_file_prefix=str(region_pref), id_digits=id_digits,
             load_vqa_ann=False)

In [7]:
PHASE_2 = Path('/home/jupyter/hate_phase2')
train = pd.read_json(PHASE_2/'train.jsonl', lines=True)
dev_seen = pd.read_json(PHASE_2/'dev_seen.jsonl', lines=True)
dev_unseen = pd.read_json(PHASE_2/'dev_unseen.jsonl', lines=True)
test_seen = pd.read_json(PHASE_2/'test_seen.jsonl', lines=True)
test_unseen = pd.read_json(PHASE_2/'test_unseen.jsonl', lines=True)
data = pd.concat([train, dev_seen, dev_unseen,test_seen,test_unseen])
data

,id,img,label,text
0,42953,img/42953.png,0.0,its their character not their color that matters
1,23058,img/23058.png,0.0,don't be afraid to love again everyone is not like your ex
2,13894,img/13894.png,0.0,putting bows on your pet
3,37408,img/37408.png,0.0,i love everything and everybody! except for squirrels i hate squirrels
4,82403,img/82403.png,0.0,"everybody loves chocolate chip cookies, even hitler"
...,...,...,...,...
1995,71352,img/71352.png,NaN,fighting for gay rights
1996,2164,img/02164.png,NaN,that feeling when you finish your homework in record time and have extra time to chill
1997,3587,img/03587.png,NaN,the day that shook new york city
1998,47839,img/47839.png,NaN,one of the first prototypes of the atom bomb


In [8]:
test_unseen[test_unseen.id==43976]

,id,img,text
656,43976,img/43976.png,when your dive instructor starts yelling in arabic


In [9]:
def get_y(row):
    return 0
def load_for_LM(row, proc, tokenizer, q=None):
    img_id = row.id
    text = row.text
    if q is None: q = text
    img_file = id_to_img_path(img_id)
    instance = (img_file, tokenizer.tokenize(q), {'answers': ['dummy']})
    input_ids, segment_ids, input_mask, lm_label_ids,   masked_pos, masked_weights, is_next, task_idx, conv_feats, vis_masked_pos, vis_pe, ans_labels= proc(instance)
    
    lm_label_ids, is_next, masked_pos,masked_weights, task_idx,vis_masked_pos = (torch.tensor(x) for x in [ lm_label_ids, is_next, masked_pos,masked_weights, task_idx,vis_masked_pos])
    return conv_feats, vis_pe, input_ids, segment_ids, input_mask, lm_label_ids, ans_labels, is_next, masked_pos,masked_weights, task_idx,vis_masked_pos

class LoadRow(Transform):
   
    def __init__(self,  processor, tokenizer):
        self.proc = processor
        self.tokenizer = tokenizer
    
    def encodes(self, x, **kwargs):
        return load_for_LM(x, self.proc, self.tokenizer)

In [10]:
db = DataBlock(blocks = (TransformBlock, CategoryBlock), 
               get_x = LoadRow(proc, tokenizer), 
               get_y=get_y,splitter = RandomSplitter(valid_pct=0.0052))
              # get_y=get_y,splitter = RandomSplitter(valid_pct=0.10))

In [11]:
dls = db.dataloaders(data,bs=64)
len(dls.train_ds), len(dls.valid_ds)

(12475, 65)

In [12]:
batch = dls.one_batch()


In [13]:
[x.shape for x in batch[0]]

[torch.Size([64, 100, 2048]),
 torch.Size([64, 100, 1607]),
 torch.Size([64, 203]),
 torch.Size([64, 203]),
 torch.Size([64, 203, 203]),
 torch.Size([64, 3]),
 torch.Size([64, 1]),
 torch.Size([64]),
 torch.Size([64, 3]),
 torch.Size([64, 3]),
 torch.Size([64]),
 torch.Size([64, 0])]

In [14]:
def fake_loss(x,y): return x[0]

In [15]:
learn = Learner(dls, new_model(),loss_func=fake_loss).to_fp16()

setting the new dropout rate! 0.25


In [17]:
learn.fit_one_cycle(10 , lr_max = 1e-4)

epoch,train_loss,valid_loss,time
0,4.310086,3.695831,05:47
1,3.523596,3.233771,04:11
2,3.174065,2.787822,04:10
3,2.890528,2.189437,04:10
4,2.695524,2.346471,04:11
5,2.515508,2.382189,04:11
6,2.421152,1.829092,04:11
7,2.274048,1.774992,04:11
8,2.197291,2.352219,04:11
9,2.204267,1.915022,04:11


In [18]:
learn.fit_one_cycle(15 , lr_max = 5e-5)

epoch,train_loss,valid_loss,time
0,2.163623,1.923067,04:11
1,2.138697,1.836306,04:11
2,2.149936,2.005157,04:11
3,2.126229,1.723796,04:11
4,2.040904,1.980285,04:11
5,2.008515,1.923516,04:11
6,1.919147,1.865721,04:11
7,1.853806,1.626781,04:11
8,1.829888,1.524760,04:11
9,1.776754,2.052647,04:11


In [19]:
#was 2.27

In [19]:
#learn.save('trained_lm20drop')
#learn.save('lm_token')
#learn.load('lm_token')
learn.save('trained_long')

Path('models/trained_long.pth')

In [16]:
learn.load('trained_long')

In [17]:
learn.fit_one_cycle(15 , lr_max = 5e-5)

epoch,train_loss,valid_loss,time
0,1.669330,1.141289,04:10
1,1.670578,1.276379,04:10
2,1.616257,1.280647,04:10
3,1.696579,1.427132,04:11
4,1.650644,1.352159,04:11
5,1.604179,1.460073,04:11
6,1.611321,1.464201,04:11
7,1.508295,1.223068,04:11
8,1.476136,1.502977,04:11
9,1.452808,1.151719,04:11


In [18]:
learn.fit_one_cycle(15 , lr_max = 5e-5)

epoch,train_loss,valid_loss,time
0,1.344204,1.632730,04:10
1,1.355910,1.482232,04:11
2,1.319746,1.831060,04:11
3,1.392142,1.821571,04:11
4,1.341137,1.299373,04:11
5,1.352200,1.271740,04:11
6,1.299891,1.297472,04:10
7,1.278458,0.995830,04:11
8,1.216862,1.630893,04:11
9,1.168598,1.138420,04:10


In [20]:
class HateStem(torch.nn.Module):
    
    def __init__(self, vlp):
        super(HateStem, self).__init__()
        self.vis_embed = vlp.vis_embed #Linear->ReLU->Linear->ReLU->dropout
        self.vis_pe_embed = vlp.vis_pe_embed #Linear->ReLU->dropout
        self.bert = vlp.bert # pytorch_pretrained_bert.modeling.BertModel
        self.len_vis_input = vlp.len_vis_input
        
    
    def forward(self, vis_feats, vis_pe, input_ids, token_type_ids=None, attention_mask=None):
        vis_feats = self.vis_embed(vis_feats) # image region features
        vis_pe = self.vis_pe_embed(vis_pe) # image region positional encodings

        sequence_output, pooled_output = self.bert(vis_feats, vis_pe, input_ids, token_type_ids,
            attention_mask, output_all_encoded_layers=False, len_vis_input=self.len_vis_input)
        #print(sequence_output.shape, pooled_output.shape)
        vqa2_embed = sequence_output[:, 0]*sequence_output[:, self.len_vis_input+1]
        return vqa2_embed
        #return sequence_output


In [21]:
lm_stem = HateStem(learn.model)

In [22]:
torch.save(lm_stem, 'checkpoints/lm_stem_long.pth')